<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [8]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [9]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [10]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [11]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [12]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\elere\AppData\Local\Temp\ipykernel_3800\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_3800\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_3800\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [13]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [14]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'], inplace=True)
apps = apps.drop_duplicates(subset='App', keep="first")

In [15]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [16]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [17]:
DataProcessing.normalization(apps)

In [18]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Creativity,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing
0,Word Crossy - A crossword game,0.875,5.355442e-03,0.052367,4.999999e-03,Everyone,0.432857,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Eshopcy.com.cy,0.800,4.031924e-06,0.003927,4.999000e-06,Everyone,0.171429,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AfterFocus,0.800,3.680478e-03,0.014099,9.999999e-03,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dr. Oetker recipe ideas,0.800,1.895227e-04,0.016113,9.999990e-04,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Yahoo! transit guide free timetable, operation...",0.850,2.334484e-03,0.021148,9.999999e-03,Everyone,0.428571,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Power Rangers: Legacy Wars,0.825,5.897613e-03,0.090634,9.999999e-03,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Weapon AK-74 Live Wallpaper,0.575,5.346197e-07,0.045317,9.990000e-07,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Teen',
 'Everyone 10+',
 'Mature 17+',
 'Adults only 18+',
 'Unrated']

In [20]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [21]:
validatingSet.sort_values(by=['Content Rating'])

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Creativity,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing
0,Word Crossy - A crossword game,0.875,5.355442e-03,0.052367,4.999999e-03,Teen,0.432857,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Eshopcy.com.cy,0.800,4.031924e-06,0.003927,4.999000e-06,Teen,0.171429,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AfterFocus,0.800,3.680478e-03,0.014099,9.999999e-03,Teen,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dr. Oetker recipe ideas,0.800,1.895227e-04,0.016113,9.999990e-04,Teen,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Yahoo! transit guide free timetable, operation...",0.850,2.334484e-03,0.021148,9.999999e-03,Teen,0.428571,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Power Rangers: Legacy Wars,0.825,5.897613e-03,0.090634,9.999999e-03,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Weapon AK-74 Live Wallpaper,0.575,5.346197e-07,0.045317,9.990000e-07,Teen,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
trainingSet.loc[trainingSet['App'] == 'BURGER KING® App']['Android Ver'] 

6361    0.442857
Name: Android Ver, dtype: float64

<h2>k-nearest neighbors algorithm</h2>

In [23]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [24]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[7:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head()

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,distance
6728,AE Jewels 2: Island Adventures,0.800,0.000015,0.029204,0.00001,Everyone,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.350182
3819,Heroes of Dragon Age,0.800,0.001568,0.027190,0.00100,Teen,0.190000,1,0,0,...,0,0,0,0,0,0,0,0,0,0.350200
6225,Counter Terrorist Gun Strike CS: Special Forces,0.800,0.000005,0.043303,0.00001,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.350241
2571,Gun Builder ELITE,0.775,0.001981,0.033233,0.00100,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.350247
4722,Gunship Modern Combat 3D,0.800,0.000072,0.025176,0.00050,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.350263


<h4>Method 2</h4>

In [25]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,distance
288,Big Hunter,0.825,0.005468,0.083585,0.01000,Everyone 10+,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.670358
1170,Real Gangster Crime,0.825,0.006367,0.083585,0.01000,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.670360
5965,Zombie Avengers:(Dreamsky)Stickman War Z,0.825,0.000303,0.095670,0.00100,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.672355
1060,Fatal Raid - No.1 Mobile FPS,0.825,0.001258,0.080564,0.00100,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.672687
3307,DEER HUNTER 2018,0.825,0.021288,0.081571,0.01000,Teen,0.285714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.673997
29,Creative Destruction,0.850,0.004487,0.079557,0.00500,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.676745
2527,Dead Target Zombie Shooting US Sniper Killer S...,0.825,0.000005,0.064451,0.00005,Teen,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,0,0.677091
6612,Rope Hero 3,0.850,0.001022,0.098691,0.00500,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.677196
6703,League of Stickman 2018- Ninja Arena PVP(Dream...,0.850,0.000724,0.098691,0.00100,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,0,0.677563
3753,Just A Regular Arcade,0.800,0.000910,0.093656,0.00100,Everyone,0.190000,1,0,0,...,0,0,0,0,0,0,0,0,0,0.677803


<h2>Importing Reviews Database</h2>

In [26]:
reviews = pd.read_csv(r"processedReviews.csv")

In [27]:
display(reviews)

,App,reviews
0,10 Best Foods for You,best;food;health;healthy;love
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy;job;use;useful;mobile
2,11st,for;english;there;language;if
3,1800 Contacts - Lens Store,easy;contacts;order;super;love
4,1LINE – One Line with One Touch,game;play;love;ads;many
...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,hotel;book;pet;friendly;find
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,trial;days;free;asking;credit
862,Hotstar,shows;answer;time;u;playing
863,Hotwire Hotel & Car Rental App,would;hotels;hotel;better;card


In [28]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [29]:
display(appsWithReviews)

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews
0,Cooking Madness - A Chef's Restaurant Games,0.925,0.007993,0.048338,0.01000,Everyone,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,0,game;get;levels;level;make
1,Garden Photo Frames - Garden Photo Editor,0.850,0.000041,0.012085,0.00050,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,photo;kumar;love;think;much
2,Hideman VPN,0.775,0.001975,0.008661,0.00500,Everyone,0.428571,0,0,0,...,0,0,0,1,0,0,0,0,0,work;works;loading;phone;connection
3,Blossom Blast Saga,0.875,0.025070,0.062437,0.01000,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,0,game;levels;lives;love;im
4,10 Best Foods for You,0.750,0.000055,0.002820,0.00050,Everyone 10+,0.190000,0,0,0,...,0,0,0,0,0,0,0,0,0,best;food;health;healthy;love
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,Doctor Pets,0.800,0.000514,0.020141,0.00100,Everyone,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,0,game;love;pets;much;animals
618,Current debit card and app made for teens,0.825,0.000015,0.020141,0.00005,Everyone,0.485714,0,0,0,...,0,0,0,0,0,0,0,0,0,money;love;card;get;current
619,"Hide Something - Photo, Video",0.900,0.005014,0.004330,0.00500,Everyone,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,0,im;hide;hidden;photos;know
620,HTC Weather,0.725,0.000493,0.036395,0.01000,Everyone,0.485714,0,0,0,...,0,0,0,0,1,0,0,0,0,location;weather;htc;city;time


In [30]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews
0,"Hide Something - Photo, Video",0.900,0.005014,0.004330,0.0050,Everyone,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,0,im;hide;hidden;photos;know
1,HTC Weather,0.725,0.000493,0.036395,0.0100,Everyone,0.485714,0,0,0,...,0,0,0,0,1,0,0,0,0,location;weather;htc;city;time
2,Baby Name Together,0.750,0.000010,0.023162,0.0001,Everyone,0.571429,0,0,0,...,0,0,0,0,0,0,0,0,0,names;love;variety;baby;able


In [31]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [32]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h4>Method 1</h4>

In [33]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[7:-1]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],maxAndroidVer]
for i in genres:
    avgAppList.append(i)
revs = []
for i in validatingSet["reviews"].tolist():
    i = i.split(";")
    for j in i:
        revs.append(j)
words = {}
for word in revs:
        words[word] = 0
for word in revs:
        words[word] +=1
revs = []
for i in range(5):
    try:
        revs.append(max(words, key=words.get))
        del words[max(words, key=words.get)]
    except:
        pass
avgWords = ";".join(revs)
avgAppList.append(avgWords)

avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNNWithJaccardSimilarityCoefficient.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head(10)

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews,distance
360,Call of Duty:Black Ops Zombies,0.800,0.000290,0.045317,0.0001,Teen,0.190000,1,0,0,...,0,0,0,0,0,0,0,0,play;game;online;doesnt;even,0.577933
428,Dinosaur Simulator: Dino World,0.750,0.000594,0.065458,0.0050,Teen,0.457143,1,0,0,...,0,0,0,0,0,0,0,0,game;many;ads;time;fun,0.580535
374,Battlelands Royale,0.850,0.002353,0.060423,0.0010,Everyone 10+,0.442857,1,0,0,...,0,0,0,0,0,0,0,0,game;battle;pass;get;would,0.581621
277,DEER HUNTER 2018,0.825,0.021288,0.081571,0.0100,Teen,0.285714,1,0,0,...,0,0,0,0,0,0,0,0,slow;uninstall;reinstall;hunt;movement,0.581791
207,DINO HUNTER: DEADLY SHORES,0.850,0.030781,0.053374,0.0100,Teen,0.285714,1,0,0,...,0,0,0,0,0,0,0,0,game;play;get;would;time,0.581917
561,Extreme Match,0.875,0.000015,0.036395,0.0001,Everyone,0.442857,1,0,0,...,0,0,0,0,0,0,0,0,game;fun;ads;love;play,0.583552
297,Arrow.io,0.875,0.007646,0.036395,0.0100,Everyone 10+,0.432857,1,0,0,...,0,0,0,0,0,0,0,0,game;play;fix;fun;please,0.583579
532,GUNSHIP BATTLE: Helicopter 3D,0.825,0.047916,0.074522,0.0500,Teen,0.432857,1,0,0,...,0,0,0,0,0,0,0,0,game;gold;connection;episode;give,0.584314
369,Dragon Hills,0.875,0.007894,0.065458,0.0100,Everyone 10+,0.442857,1,0,0,...,0,0,0,0,0,0,0,0,game;time;fun;level;play,0.585055
593,Anger of stick 5 : zombie,0.875,0.012230,0.022155,0.0500,Teen,0.185714,1,0,0,...,0,0,0,0,0,0,0,0,game;cool;keeps;engaged;keep,0.585157


<h4>Method 2</h4>

In [34]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews,distance
443,Family GPS tracker KidControl + GPS by SMS Loc...,0.875,0.001273,0.036395,0.0010,Everyone,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,work;kids;phone;time;update,0.866185
78,All Social Networks,0.800,0.000501,0.000504,0.0010,Everyone,0.428571,0,0,0,...,0,0,0,0,0,0,0,0,ads;every;time;get;pop,0.882960
252,Free live weather on screen,0.850,0.000342,0.020141,0.0010,Everyone,0.442857,0,0,0,...,0,0,0,1,0,0,0,0,weather;love;ads;best;keep,0.883499
132,Couple - Relationship App,0.750,0.000741,0.007452,0.0010,Everyone,0.185714,0,0,0,...,0,0,0,0,0,0,0,0,would;love;also;couples;little,0.898782
207,DINO HUNTER: DEADLY SHORES,0.850,0.030781,0.053374,0.0100,Teen,0.285714,1,0,0,...,0,0,0,0,0,0,0,0,game;play;get;would;time,0.938486
561,Extreme Match,0.875,0.000015,0.036395,0.0001,Everyone,0.442857,1,0,0,...,0,0,0,0,0,0,0,0,game;fun;ads;love;play,0.939461
369,Dragon Hills,0.875,0.007894,0.065458,0.0100,Everyone 10+,0.442857,1,0,0,...,0,0,0,0,0,0,0,0,game;time;fun;level;play,0.940083
314,Dude Perfect 2,0.875,0.008942,0.069486,0.0100,Everyone,0.432857,1,0,0,...,0,0,0,0,0,0,0,0,game;love;perfect;play;get,0.940455
516,Alto's Adventure,0.900,0.011487,0.062437,0.0100,Everyone,0.428571,1,0,0,...,0,0,0,0,0,0,0,0,game;time;get;playing;ads,0.942085
476,Agar.io,0.800,0.085022,0.031219,0.1000,Everyone,0.432857,1,0,0,...,0,0,0,0,0,0,0,0,game;play;lag;people;time,0.943714


<h2>KD Tree Algorithm</h2>

In [35]:
class Node:
    def __init__(self, point, left=None, right=None, name=None):
        self.point = point
        self.left = left
        self.right = right
        self.name = name


class KDTree:
    def __init__(self, points):
        self.k = 4  # Number of dimensions
        self.root = self.build_tree(points)

    def build_tree(self, points, depth=0):
        if len(points) == 0:
            return None

        # Select axis based on depth
        axis = depth % self.k

        # Sort points based on the axis
        sorted_points = points.sort_values(by=points.columns[axis+2])

        # Find the median
        median_index = len(sorted_points) // 2
        median = sorted_points.iloc[median_index]
        
        # Recursively build subtrees
        left_points = sorted_points[:median_index]
        right_points = sorted_points[median_index + 1:]

        left = self.build_tree(left_points, depth + 1)
        right = self.build_tree(right_points, depth + 1)
        return Node(median[1:5], left, right, median[0])

    def find_nearest_neighbors(self, query_point, k=1):
        nearest_points = []
        distances = []
        nearest_names = []

        def search(node, depth=0):
            if node is None:
                return

            # Select axis based on depth
            axis = depth % self.k

            # Traverse left or right subtree based on query point
            if query_point[axis] < node.point[axis]:
                search(node.left, depth + 1)
            else:
                search(node.right, depth + 1)
            # Calculate distance between query point and current point
            distance = np.linalg.norm(np.array(query_point) - np.array(node.point))
            # If the number of nearest points is less than k, add current point
            if len(nearest_points) == 0:
                distances.append(distance)
                nearest_points.append(node.point)
                nearest_names.append(node.name)
            
            elif len(nearest_points) < k:
                index = len(nearest_points)
                for i in range(0, len(nearest_points)):
                    if(distance < distances[i]):
                        index = i
                        break
                distances.insert(index, distance)
                nearest_points.insert(index, node.point)
                nearest_names.insert(index, node.name)
            else:
                # If the current point is closer than the farthest nearest point,
                # replace the farthest point with the current point
                if distance < distances[-1]:
                    for i in range(0, k):
                        if(distance < distances[i]):
                            distances.insert(i, distance)
                            nearest_points.insert(i, node.point)
                            nearest_names.insert(i, node.name)
                            break
                    distances.pop()
                    nearest_points.pop()
                    nearest_names.pop()
            # Check if there can be closer points on the other side of the splitting axis
            if abs(query_point[axis] - node.point[axis]) < distances[-1] or len(nearest_points) < k:
                if query_point[axis] < node.point[axis]:
                    search(node.right, depth + 1)
                else:
                    search(node.left, depth + 1)
        search(self.root)
        return nearest_names
    

<h2>Divide apps dataset by genres</h2>

In [56]:
def divide_by_genres(dataset):
    datasets = []
    for i in range(8, dataset.shape[1]):
        datasets.append(dataset[dataset.iloc[:, i] == 1])
    return datasets
trainingSet_genres = divide_by_genres(trainingSet)
trainingSet_genres[51].head()
    

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews
16,Grim Soul: Dark Fantasy Survival,0.875,0.004227,0.094663,0.005,Teen,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,1,game;get;play;playing;im
148,Equestria Girls,0.825,0.008745,0.052367,0.010,Everyone,0.428571,0,0,0,...,0,0,0,0,0,0,0,0,1,game;make;please;love;im
194,FINAL FANTASY BRAVE EXVIUS,0.900,0.016611,0.082578,0.005,Teen,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,1,game;get;play;games;playing
254,Chapters: Interactive Stories,0.875,0.001638,0.095670,0.001,Mature 17+,0.471429,0,0,0,...,0,0,0,0,0,0,0,0,1,game;choices;stories;gems;get
325,Dungeon Hunter Champions: Epic Online Action RPG,0.800,0.000585,0.036395,0.001,Teen,0.432857,0,0,0,...,0,0,0,0,0,0,0,0,1,game;play;time;get;games


<h2>Building the KD Tree</h2>

In [50]:
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Strategy,Puzzle,...,Sports,Word,Brain Games,Tools,Weather,Simulation,Maps & Navigation,Auto & Vehicles,Role Playing,reviews
0,"Hide Something - Photo, Video",0.900,0.005014,0.004330,0.0050,Everyone,0.442857,0,0,0,...,0,0,0,0,0,0,0,0,0,im;hide;hidden;photos;know
1,HTC Weather,0.725,0.000493,0.036395,0.0100,Everyone,0.485714,0,0,0,...,0,0,0,0,1,0,0,0,0,location;weather;htc;city;time
2,Baby Name Together,0.750,0.000010,0.023162,0.0001,Everyone,0.571429,0,0,0,...,0,0,0,0,0,0,0,0,0,names;love;variety;baby;able


In [57]:
#kd tree tu działa tak, że dla każdej kategorii tworzy osobne drzewo, a następnie dla każdej kategorii występującej w podanej aplikacji wyszukuje 5 najbliższych sąsiadów
#następnie łączy najbliższych sąsiadów z validatingSet i liczy dystans używając jaccard similarity coefficient, i na tej podstawie zwraca 5 najbliższych sąsiadów
kdtrees = []
for i in range(len(trainingSet_genres)):
    kdtrees.append(KDTree(trainingSet_genres[i]))
def recommendation_genres(dataset, app):
    app_genres = []
    recommended_apps = []
    for i in range(8, dataset.shape[1]-1):
        if app[i] == 1:
            app_genres.append(i-8)
    for i in range(len(app_genres)):
        recommended_apps_genre = kdtrees[app_genres[i]].find_nearest_neighbors(app[1:5].to_numpy(), 5)
        recommended_apps.extend(x for x in recommended_apps_genre if x not in recommended_apps)
    return recommended_apps
def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
def kdtree_recommendation(dataset, validatingApps):
    recommended_apps = []
    for idx, row in validatingApps.iterrows():
        recommended_apps.extend(x for x in recommendation_genres(dataset, row) if x not in recommended_apps)
    distances = []
    for idx, i in enumerate(recommended_apps):
        distances.append(0)
        for _, j in validatingApps.iterrows():
            distances[idx] += (np.linalg.norm(np.array(dataset[dataset["App"] == i].iloc[0][1:5]) - np.array(j[1:5])))
            distances[idx] *= jaccard(dataset[dataset["App"] == i].iloc[0][-1], j[-1])
    best_apps = []
    for i in range(5):
        best_apps.append(recommended_apps[distances.index(min(distances))])
        distances.remove(min(distances))
        recommended_apps.remove(best_apps[-1])
    return best_apps

kdtree_recommendation(trainingSet, validatingSet)

['Baby Panda Musical Genius',
 'Baby Panda Learns Shapes',
 'ForecaWeather',
 'APE Weather ( Live Forecast)',
 'Amazon FreeTime – Kids’ Videos, Books, & TV shows']